# DW PROJECT - PART A

In [ ]:
'''
#Authors: Lital Hendizadeh, Edan Shahmoon and Shai Ardazi
'''

## Importing required packages

In [ ]:
import petl as etl
import MySQLdb as mdb
import csv
from datetime import timedelta, date
import holidays
import calendar
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import movie_reviews
from nltk.corpus import stopwords
import pickle
import os.path
from datetime import datetime
from tqdm import tqdm_notebook as tqdm

### Words classification may take a lot of time. Please change variable "debug" to False only when necessary

In [ ]:
##########################
debug=True
##########################

## Extract - Reading from files

In [ ]:
business_json = etl.fromjson('yelp_academic_dataset_business_PA.json')
users_json = etl.fromjson('yelp_academic_dataset_users_nofriendlist_PA.json')
reviews_json_facts = etl.fromjson('yelp_academic_dataset_review_drop_PA.json')
full_reviews_good=etl.fromcsv('Pittsburgh_full_reviews_text.csv',encoding='UTF-8')

## Transform - Types convertions functions

In [ ]:

#Convertion functions
def to_string(text):
    return str(text)
def to_float(text):
    return float(text)
def to_int(text):
    return int(text)
def to_boolean(text):
    if text =="True":
        return True
    if text == "False":
        return False

  
def to_list (text):
    if text[0]=='[' and text[-1]==']':
        items = text[2:-1].split(', u') # turn values to list
    else:
        return None
    for i in range(len(items)):
        items[i]=items[i][1:-1]
    return items
def to_date (text):
    return datetime.strptime(text, '%Y-%m-%d')

#Functions that call the convertions functions
def converting_to_str(val,name_of_tbl1):
    return name_of_tbl1.convert(val,to_string)
def converting_to_int(val,name_of_tbl1):
    return name_of_tbl1.convert(val,to_int)
def converting_to_float(val,name_of_tbl1):
    return name_of_tbl1.convert(val,to_float)
def converting_to_list(val,name_of_tbl1):
    return name_of_tbl1.convert(val,to_list)
def converting_to_date(val,name_of_tbl1):
    return name_of_tbl1.convert(val,to_date)
def converting_to_boolean(val,name_of_tbl1):
    return name_of_tbl1.convert(val,to_boolean) 


def to_dict(text):
    if text[0]=='[' and text[-1]==']':
        items = text[1:-1].split(', ') # turn values to list
    else:
        return None
    output = {}
    for i in items:
        key, value = i.split(':') # splitting to key and value
        output[key]=int(value)
    return items

## Fields calculation functions 

In [ ]:
#global variables required for the tables' ids creation
global_user_id=0
global_bussiness_id2=0
global_review_id=0
global_date_id=0

#Creating ids for tables
def bussiness_id_generator (row):
    global global_bussiness_id2
    global_bussiness_id2+=1
    return global_bussiness_id2

def date_id_generator (row):
    global global_date_id
    global_date_id+=1
    return global_date_id


def user_id_generator (row):
    global global_user_id
    global_user_id+=1
    return global_user_id

def more_than_20 (row):#checking if a business has more than 20 reviews since 20 is the limit for reviews in a business page
    if row['review_count']>20:
        return True
    else:
        return False
    
#user table

def active_years (row):#user's active years
    yelping_start_year=row["yelping_since"].year
    #cur_year = datetime.datetime.now.year
    cur_year=2017
    return cur_year-yelping_start_year

def funny_average(row):
    if row['review_count']==0:
        return 0
    return float(row['funny'])/row['review_count']

def cool_average(row):
    if row['review_count']==0:
        return 0
    return float(row['cool'])/row['review_count']

def fans_average (row):
    if row["fans"]==0:
        return 0
    return row["fans"]/float(row["review_count"])

def useful_average(row):
    if row['review_count']==0:
        return 0
    return float(row['useful'])/row['review_count']

def weighted_score_for_reviews (row):
    useful=row["useful"]
    funny=row["funny"]
    cool=row["cool"]
    weighted_mean=(0.5*useful+0.25*(funny+cool))/float(12)
    return weighted_mean

def yelp_celebrity (row):#Based on pre calculations on our data only- there are several hundrenrds users who have more than 50 followers
    return row["fans"]>50

def loveliness_level (row):
    avg_stars=row['average_stars']
    if avg_stars<1.5:
        return "1" #disgrading user
    elif 1.5<avg_stars<3.5:
        return "2"#average user
    return"3"#a giver user

def active_user (row):
    return row["review_count"]>1000

def elite_first_year_func (row):
    elite_first_year=min(row["elite"])
    return elite_first_year

def elite_counter_func (row):
    return len(row["elite"])

def is_elite_gold_func (row):
    if len(row['elite'])>5:
        return True
    else:
        return False
    
def is_elite_black_func (row):
    if len(row['elite'])>10:
        return True
    else:
        return False
    
def num_of_compliment_func (row):
    sum_com= row["compliment_cute"]+row["compliment_writer"]+row["compliment_note"]+row["compliment_cool"]+row["compliment_profile"]+row["compliment_hot"]+row["compliment_more"]+row["compliment_plain"]+row["compliment_funny"]+row["compliment_list"]+row["compliment_photos"]
    return sum_com

def time_to_elite_func (row):
    elite_first_year=row["elite_first_year"]
    if elite_first_year == "None" :
      return -1
    elite_first_year=int (row["elite_first_year"])
    yelping_start_year=row["yelping_since"].year
    ans=elite_first_year-yelping_start_year
    return ans

def elite_sequence(row):
    return 999

#fact table

def review_id_generator(row):
    global global_review_id
    global_review_id+=1
    return global_review_id

def ratio_funny_review_user(row):
    if row["funny_average"] == 0:  
        return 0
    return float(row['funny']) / row["funny_average"]

def ratio_cool_review_user(row):
    if row["cool_average"] == 0:  
        return 0
    return float(row['cool']) / row["cool_average"]

def ratio_useful_review_user(row):
    if row["useful_average"] == 0:  
        return 0
    return float(row['useful']) / row["useful_average"]

def ratio_funny_reactios(row):
    if row["funny"] == 0:  
        return 0
    return float(row['funny']) / (row["funny"]+row['cool']+row['useful'])

def ratio_cool_reactios(row):
    if row["cool"] == 0:  
        return 0
    return float(row['cool']) / (row["funny"]+row['cool']+row['useful'])
def ratio_useful_reactios(row):
    if row["useful"] == 0:  
        return 0
    return float(row['useful']) / (row["funny"]+row['cool']+row['useful'])

def distance_stars_user(row): 
    return float(row['r_stars'])-float(row['average_stars'])

def distance_stars_business(row): 
    return float(row['r_stars'])-float(row['b_stars'])

def sentimemt_func(row):
    return classify(row["text"])[1]

def folaritey_func(row):
    return classify(row["text"])[2]
def num_of_words_in_review_func(row):
    if row["text"]==None:
        return 0
    return len(row["text"])



## Creat Date Dimension

In [ ]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)
#these dates were chosen in order to be able to have all existing dates indexed in our dates table.
start_date = date(2004, 1, 1)
end_date = date(2018, 6, 2)
dates=[['date_id','full_date','day','day_name','month','month_name','year','is_weekend','is_holiday','holiday_name']]
us_holidays = holidays.UnitedStates()
month_lst=[0,'January','February','March','April','May','June','July','August','September','October','November','December']
date_id=0

for single_date in daterange(start_date, end_date):
    date_id+=1
    full_date=single_date
    year=full_date.year
    month=full_date.month
    day=full_date.day
    is_holiday=full_date in us_holidays
    holiday_name=us_holidays.get(full_date)
    day_name=calendar.day_name[calendar.weekday(year, month, day)]
    month_name = month_name = month_lst[month]
    is_weekend=day_name in ['Saturday','Sunday']
    
    dates.append([date_id,full_date,day,day_name,month,month_name,year,is_weekend,is_holiday,holiday_name])
with open('dates_dim.csv', 'wb') as f:
        writer = csv.writer(f)
        writer.writerows(dates)
dates_dim = etl.fromcsv('dates_dim.csv')


## Converting fields to required types

In [ ]:

def types_convertion(business_json1,users_json1,reviews_json_facts1,full_reviews,dates_dim1):
    counter=0
    f_to_str_b=["city","name","state","address","business_id"]#columns in business table  we convert to str
    f_to_float_b=["stars"]#column in business table we convert to boolean
    f_to_list_b=["categories","hours","attributes"]#columns in business table we convert to lists
    f_to_str_u=["user_id","name","yelping_since"]#columns in users table we convet to string
    f_to_str_facts=["user_id","review_id","business_id","date"]#columns in fact table we convert to string
    full_review_to_str=["review_id","text"]#columns in full reviews table we convert to string
    d_to_str=["day_name","month_name","holiday_name"]#columns in dates table we convert to string
    d_to_int=["date_id","day","month","year"]#columns in dates table we convert to integer
    d_to_bool=["is_weekend","is_holiday"]#columns in dates table we convert to boolean
    
    #converting to required types
    for field in f_to_str_b:
        f=field
        business_json1=converting_to_str(f,business_json1)
    for field in f_to_float_b:
        f=field
        business_json1=converting_to_float(f,business_json1)
    for field in f_to_list_b:
        f=field
        business_json1=converting_to_list(f,business_json1)
    for field in f_to_str_u:
        f=field
        users_json1=converting_to_str(f,users_json1)
    for field in f_to_str_facts:
        f=field
        reviews_json_facts1=converting_to_str(f,reviews_json_facts1)
    for field in full_review_to_str:
        f=field
        full_reviews=converting_to_str(f,full_reviews)
    for field in d_to_str:
        f=field
        dates_dim1=converting_to_str(f,dates_dim1)
    for field in d_to_bool:
        f=field
        dates_dim1=converting_to_boolean(f,dates_dim1)
    for field in d_to_int:
        f=field
        dates_dim1=converting_to_int(f,dates_dim1)

    #three convertions that are not part of the for loops:
    users_json1=converting_to_list("elite",users_json1)
    users_json1=converting_to_date("yelping_since",users_json1)
    reviews_json_facts1=converting_to_date("date",reviews_json_facts1) 
    dates_dim1=converting_to_date("full_date",dates_dim1) 
    return business_json1,users_json1,reviews_json_facts1,full_reviews,dates_dim1

#Calling the convertions function
b,u,r,fv,d=types_convertion(business_json,users_json,reviews_json_facts,full_reviews_good,dates_dim)


## Displaying all converted tables

In [ ]:
b.display(1)
u.display(1)
r.display(1)
d.display(1)

## Checking all types of columns in all converted tables

In [ ]:
fields2 = b.fieldnames()
for f in tqdm(fields2):
    print f,'\t', b.typecounter(f)
    
fields2 = u.fieldnames()
for f in tqdm(fields2):
    print f,'\t', u.typecounter(f)
    
fields2 = r.fieldnames()
for f in tqdm(fields2):
    print f,'\t', r.typecounter(f)
    
fields2 = d.fieldnames()
for f in tqdm(fields2):
    print f,'\t', d.typecounter(f)

## Transform (cont) - adding fields based on calculations we did before 

In [ ]:
#Adding fields to business table
b = b.addfield('more_than_20',more_than_20)
b = b.addfield('business_id_n',bussiness_id_generator)
b = b.rename({'stars':'b_stars'})

#Adding fields to users table
u = u.addfield('active_years',active_years)
u = u.addfield('elite_first_year',elite_first_year_func)
u = u.addfield('time_to_elite',time_to_elite_func)
u = u.addfield('is_elite_gold',is_elite_gold_func)
u = u.addfield('is_elite_black',is_elite_black_func)
u = u.addfield('elite_counter',elite_counter_func)
u = u.addfield('active_years',active_years)
u = u.addfield('elite_sequence',elite_sequence)
u = u.addfield('weighted_score_for_reviews',weighted_score_for_reviews)
u = u.addfield('yelp_celebrity',yelp_celebrity)
u = u.addfield('loveliness_level',loveliness_level)
u = u.addfield('active_user',active_user) 
u = u.addfield('funny_average',funny_average)
u = u.addfield('cool_average',cool_average)
u = u.addfield('fans_average',fans_average)
u = u.addfield('user_id_n',user_id_generator)
u = u.addfield('num_of_compliment',num_of_compliment_func)

#converting user_id_n to int
field='user_id_n'
u=converting_to_int(field,u)

#Adding fields to reviews table
r = r.addfield('ratio_funny_reactios',ratio_funny_reactios)
r = r.addfield('ratio_cool_reactios',ratio_cool_reactios)
r = r.addfield('ratio_useful_reactios',ratio_useful_reactios)
r = r.addfield('review_id_n',review_id_generator)
r = r.rename({'stars':'r_stars'})

#Join reviews and users tables in order to add more fields based on this join
r_lj_u=etl.leftjoin(r,u, key='user_id')
r_lj_u = r_lj_u.addfield('funny_average',funny_average)
r_lj_u = r_lj_u.addfield('cool_average',cool_average)
r_lj_u = r_lj_u.addfield('useful_average',useful_average)
r_lj_u = r_lj_u.addfield('ratio_funny_review_user',ratio_funny_review_user)
r_lj_u = r_lj_u.addfield('ratio_cool_review_user',ratio_cool_review_user)
r_lj_u = r_lj_u.addfield('ratio_useful_review_user',ratio_useful_review_user)
r_lj_u = r_lj_u.addfield('distance_stars_user',distance_stars_user)


#Join reviews and business tables in order to add more fields based on this join
r_lj_b=etl.leftjoin(r,b, key='business_id')
r_lj_b=r_lj_b.addfield('distance_stars_business',distance_stars_business)



## Checking joined tables

In [ ]:
r_lj_u.display(1)
r_lj_b.display(1)

## Cutting required columns

In [ ]:
#creating our final tables for the dimentions - users and business
users_dim=u.cut(['user_id_n','user_id','name','useful','funny','cool','weighted_score_for_reviews','fans','yelp_celebrity','loveliness_level','average_stars','yelping_since','compliment_photos','compliment_list','compliment_funny','compliment_plain','compliment_note','compliment_writer','compliment_cute','review_count','compliment_more','compliment_hot','elite','compliment_profile','compliment_cool','active_years','elite_first_year','time_to_elite','is_elite_gold','is_elite_black','elite_sequence','active_user','funny_average','cool_average','fans_average','elite_counter','num_of_compliment'])
business_dim=b.cut(['business_id_n','city', 'review_count', 'name', 'business_id','is_open', 'b_stars', 'address','more_than_20'])

#creating our fact table
temp_r_lj_u=r_lj_u.cut(['review_id_n','user_id_n','user_id','business_id','review_id','date','r_stars','funny','cool', 'useful','ratio_funny_review_user', 'ratio_cool_review_user', 'ratio_useful_review_user', 'distance_stars_user'])
temp_r_lj_b=r_lj_b.cut(["review_id",'business_id_n',"distance_stars_business"])
fact_table=etl.join(temp_r_lj_u,temp_r_lj_b, key='review_id')


## Text characterization

In [ ]:
# list of stopwords adopted from nltk
stopset = ['all', 'just', 'being', 'over', 'both', 'through', 'yourselves', 'its', 'before', 'o', 'hadn', 'herself', 'll', 'had', 'should', 'to', 'only', 'won', 'under', 'ours', 'has', 'do', 'them', 'his', 'very', 'they', 'not', 'during', 'now', 'him', 'nor', 'd', 'did', 'didn', 'this', 'she', 'each', 'further', 'where', 'few', 'because', 'doing', 'some', 'hasn', 'are', 'our', 'ourselves', 'out', 'what', 'for', 'while', 're', 'does', 'above', 'between', 'mustn', 't', 'be', 'we', 'who', 'were', 'here', 'shouldn', 'hers', 'by', 'on', 'about', 'couldn', 'of', 'against', 's', 'isn', 'or', 'own', 'into', 'yourself', 'down', 'mightn', 'wasn', 'your', 'from', 'her', 'their', 'aren', 'there', 'been', 'whom', 'too', 'wouldn', 'themselves', 'weren', 'was', 'until', 'more', 'himself', 'that', 'but', 'don', 'with', 'than', 'those', 'he', 'me', 'myself', 'ma', 'these', 'up', 'will', 'below', 'ain', 'can', 'theirs', 'my', 'and', 've', 'then', 'is', 'am', 'it', 'doesn', 'an', 'as', 'itself', 'at', 'have', 'in', 'any', 'if', 'again', 'no', 'when', 'same', 'how', 'other', 'which', 'you', 'shan', 'needn', 'haven', 'after', 'most', 'such', 'why', 'a', 'off', 'i', 'm', 'yours', 'so', 'y', 'the', 'having', 'once']

def word_feats(words):
    return dict([(word, True) for word in words if word not in stopset])

def save_classifier(classifier):
   f = open('trained_classifier.pickle', 'wb')
   pickle.dump(classifier, f, -1)
   f.close()

def load_classifier():
   f = open('trained_classifier.pickle', 'rb')
   classifier = pickle.load(f)
   f.close()
   return classifier

#training the classifier to be able to identify the text
def train_new_classifier():
    '''
    create a Naive Bayes Classifier based on movie reviews corpus
    :return: 
    NaiveBayesClassifier
    '''
    negids = movie_reviews.fileids('neg')
    posids = movie_reviews.fileids('pos')

    negfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'neg') for f in negids]
    posfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'pos') for f in posids]

    trainfeats = negfeats + posfeats
    testfeats = negfeats + posfeats
    
    classifier = NaiveBayesClassifier.train(trainfeats)
    save_classifier(classifier)
    return classifier


In [ ]:
#Checks if the classifier file exists in the same folder. if not, it creates one (may take some time)
if os.path.exists('trained_classifier.pickle'):
    classifier=load_classifier()
else:
    classifier=train_new_classifier()

In [ ]:
def classify(d):
    if debug: #if debug == True --> it will return 0,0,0 (In order to work quickly)
        return 0,0,0
    #
    # if debug == False --> it will calculate all reviews' texts (may take a lot of time!)
    #
    if d==None:
        return d,'na',0
    a=classifier.prob_classify(word_feats(d))._prob_dict
    sentiment=classifier.classify(word_feats(d))
    magnitude=abs(float(a[sentiment])/(abs(a['pos'])+abs(a['neg'])))
    return d,sentiment,magnitude

# Joining tables

In [ ]:
#Creating our final users table (including date id)
u_lj_dates=etl.leftjoin (users_dim,d,lkey='yelping_since',rkey='full_date')
users_dim=u_lj_dates.cut(['user_id_n','user_id','name','useful','funny','cool','weighted_score_for_reviews','fans','yelp_celebrity','loveliness_level','average_stars','date_id','compliment_photos','compliment_list','compliment_funny','compliment_plain','compliment_note','compliment_writer','compliment_cute','review_count','compliment_more','compliment_hot','elite','compliment_profile','compliment_cool','active_years','elite_first_year','time_to_elite','is_elite_gold','is_elite_black','elite_sequence','active_user','funny_average','cool_average','fans_average','elite_counter','num_of_compliment'])
users_dim = users_dim.rename({'date_id':'yelping_since(date_id)'})

####################

fv = fv.addfield('sentimemt',sentimemt_func)
fv = fv.addfield('magnitude',folaritey_func)
fv = fv.addfield('num_of_words_in_review',num_of_words_in_review_func)
fv2= fv.cut(['review_id','magnitude','sentimemt','num_of_words_in_review'])

#Creating our final fact table (including date id and text charictrization fields - sentiment, magnitude and num of words in a review)
fact_lj_fv1=etl.join(fact_table,fv2,key='review_id')
r_lj_dates_dim=etl.leftjoin(fact_lj_fv1,d, lkey='date',rkey='full_date')
fact_table1=r_lj_dates_dim.cut(['review_id_n','user_id','user_id_n','business_id','review_id','date_id','r_stars','funny','cool','useful','ratio_funny_review_user','ratio_cool_review_user','ratio_useful_review_user','distance_stars_user','business_id_n','distance_stars_business','magnitude','sentimemt','num_of_words_in_review'])


## Last convertion in fact table

In [ ]:
fact_table1=converting_to_float("magnitude",fact_table1)
fact_table1=converting_to_int("num_of_words_in_review",fact_table1)
fact_table1=converting_to_str("sentimemt",fact_table1)


## Displaying final tables

In [ ]:
business_dim.display(6)
users_dim.display(6)
fact_table.display(6)
d.display(6)

## Connecting to DB

In [ ]:
#con = mdb.connect(host = 'yelpdb.db.14017742.hostedresource.com', user = 'yelpdb', passwd = 'Sha!ardez1') 
con = mdb.connect(host = 'localhost', user = 'root', passwd = 'root')
cursor = con.cursor()
cursor.execute("SHOW DATABASES") # running SQL
#cursor.execute('DROP SCHEMA IF EXISTS shai1')
cursor.execute('CREATE SCHEMA IF NOT EXISTS local_db')
cursor.execute("USE local_db")
cursor.execute('SET SQL_MODE=ANSI_QUOTES')



## Loading tables to DB

#### Fact table (based on reviews table)

In [ ]:
fact_table1.todb(cursor,'fact_table',schema='local_db',create='True')

#### Business

In [ ]:
business_dim.todb(cursor,'business',schema='local_db',create='True')

#### Users

In [ ]:
def change_format(file1,file2): # changing format of csv file in order to load properly to MYsqlDB
 #change from given dialect to standart
 ifile = open(file1, "rb")
 dialect = csv.Sniffer().sniff(ifile.read(1024))
 ifile.seek(0) #go back to start
 reader = csv.reader(ifile, dialect)
 ofile = open(file2, "wb")
 writer = csv.writer(ofile)
 writer.writerows(reader)
 ifile.close()
 ofile.close()

users_dim.tocsv('users_dim.csv')
change_format('users_dim.csv','users_dim_good.csv')
users_dim=etl.fromcsv('users_dim_good.csv')
users_dim.todb(cursor,'users',schema='local_db',create='True')



#### Dates

In [ ]:
d.todb(cursor,'dates',schema='local_db',create='True')

# sql configuration

In [ ]:
#PRIMARY KEYS
cursor.execute("ALTER TABLE fact_table ADD PRIMARY KEY (review_id_n);")
cursor.execute("ALTER TABLE dates ADD PRIMARY KEY (date_id);")
cursor.execute("ALTER TABLE users ADD PRIMARY KEY (user_id_n);")
cursor.execute("ALTER TABLE business ADD PRIMARY KEY (business_id_n);")
#FOREIGN KEYS
cursor.execute("ALTER TABLE fact_table ADD FOREIGN KEY (date_id) REFERENCES dates(date_id);")
cursor.execute("ALTER TABLE fact_table ADD FOREIGN KEY (business_id_n) REFERENCES business(business_id_n);")
cursor.execute("ALTER TABLE fact_table ADD FOREIGN KEY (user_id_n) REFERENCES users(user_id_n);")

cursor.commit()